In [1]:
import requests
import pandas as pd
import json
from copy import copy

In [2]:
addr = 'http://127.0.0.1:8000/'
endpoint = f"{addr}/geocoding/api/v1/address/"

### Get data

In [3]:
headers = {'Content-Type': 'text/csv'}
response = requests.post(endpoint,
                         data=open('city.csv' ,'rb'),
                         headers=headers)
response.status_code

206

In [4]:
response_data = json.loads(response.content)

In [5]:
len(response_data['errors'])

104

In [6]:
response_data['errors'][0]

{'INVALID_DATA': {'foundation_year': ['Введите правильное число.']},
 'address': {'address': 'Архангельская обл, г Мезень',
  'postal_code': '164750',
  'country': 'Россия',
  'federal_district': 'Северо-Западный',
  'region_type': 'обл',
  'region': 'Архангельская',
  'area_type': 'р-н',
  'area': 'Мезенский',
  'city_type': 'город',
  'city': 'Мезень',
  'settlement_type': None,
  'settlement': None,
  'kladr_id': '2901200100000',
  'fias_id': '8a49f67f-6c48-4160-bd1d-ebc188a69269',
  'fias_level': '4',
  'capital_marker': '1',
  'okato': '11242501000',
  'oktmo': '11642101001',
  'tax_office': '2932',
  'timezone': 'UTC+3',
  'geo_lat': '65.8398078',
  'geo_lon': '44.2532273',
  'population': '3599',
  'foundation_year': 'XVI век'}}

In [7]:
# Все ли ошибки с валидацией?
sum('INVALID_DATA' in item for item in response_data['errors'])

104

In [8]:
data = []
for item in response_data['errors']:
    address = copy(item['address'])
    error_fields = {f"_{field}":True for field in item['INVALID_DATA'].keys()}
    address.update(error_fields)
    data.append(address)
data = pd.DataFrame(data)
data.columns

Index(['address', 'postal_code', 'country', 'federal_district', 'region_type',
       'region', 'area_type', 'area', 'city_type', 'city', 'settlement_type',
       'settlement', 'kladr_id', 'fias_id', 'fias_level', 'capital_marker',
       'okato', 'oktmo', 'tax_office', 'timezone', 'geo_lat', 'geo_lon',
       'population', 'foundation_year', '_foundation_year', '_population'],
      dtype='object')

In [9]:
print(sum(data.foundation_year.isna()), sum(data.population.isna()))

0 0


Ошибки только в полях: foundation_year и population

### population

In [10]:
data[data._population == True]

,address,postal_code,country,federal_district,region_type,region,area_type,area,city_type,city,...,okato,oktmo,tax_office,timezone,geo_lat,geo_lon,population,foundation_year,_foundation_year,_population
18,"Респ Ингушетия, г Сунжа",386200,Россия,Северо-Кавказский,Респ,Ингушетия,None,None,город,Сунжа,...,26230835001,26720000001,603,UTC+3,43.3204196,45.0476331,64493[3],1845,NaN,True
44,"Ленинградская обл, Всеволожский р-н, г Кудрово",188691,Россия,Северо-Западный,обл,Ленинградская,р-н,Всеволожский,город,Кудрово,...,41212812002,41612155005,4703,UTC+3,59.9075226,30.5121008,13501[4],1926,NaN,True
45,"Ленинградская обл, Всеволожский р-н, г Мурино",188662,Россия,Северо-Западный,обл,Ленинградская,р-н,Всеволожский,город,Мурино,...,41212000101,41612428101,4703,UTC+3,60.044862,30.4571456,19775[4],1676,NaN,True
86,"Респ Татарстан, Верхнеуслонский р-н, г Иннополис",420500,Россия,Приволжский,Респ,Татарстан,р-н,Верхнеуслонский,город,Иннополис,...,92220509000,92620109001,1683,UTC+3,55.7521699,48.7446846,96[3],2012,NaN,True
87,"Респ Татарстан, г Кукмор",422110,Россия,Приволжский,Респ,Татарстан,р-н,Кукморский,город,Кукмор,...,92233501000,92633101001,1675,UTC+3,56.1861392,50.8970238,17700[4],1741,NaN,True


In [11]:
data.population.apply(lambda s: s.split('[')[0] if '[' in s else s)[data._population == True]

18    64493
44    13501
45    19775
86       96
87    17700
Name: population, dtype: object

In [12]:
data.population = data.population.apply(lambda s: s.split('[')[0] if '[' in s else s)
data.drop(columns='_population', inplace=True)

### foundation_year

In [14]:
data[data._foundation_year == True].sample(5)

,address,postal_code,country,federal_district,region_type,region,area_type,area,city_type,city,...,capital_marker,okato,oktmo,tax_office,timezone,geo_lat,geo_lon,population,foundation_year,_foundation_year
37,"Респ Крым, г Керчь",298300,Россия,Южный,Респ,Крым,None,None,город,Керчь,...,0,35412000000,35715000001,9111,UTC+3,45.3562627,36.4674513,147033,7 — 6 век до н. э.,True
92,"Тульская обл, Киреевский р-н, г Болохово",301280,Россия,Центральный,обл,Тульская,р-н,Киреевский,город,Болохово,...,0,70228508000,70628108001,7148,UTC+3,54.0820349,37.826724,9619,XVI век,True
49,"Московская обл, г Дзержинский",140090,Россия,Центральный,обл,Московская,None,None,город,Дзержинский,...,0,46411000000,46711000,5027,UTC+3,55.6240869,37.8440276,47125,1380-е,True
3,"Респ Башкортостан, г Янаул",452800,Россия,Приволжский,Респ,Башкортостан,р-н,Янаульский,город,Янаул,...,1,80460000000,80659101,264,UTC+5,56.2650146,54.929907,26988,1730-е,True
33,"Красноярский край, г Шарыпово",662310,Россия,Сибирский,край,Красноярский,None,None,город,Шарыпово,...,0,4440000000,4740000,2459,UTC+7,55.5389739,89.1801044,38570,XVIII век,True


In [15]:
import re
roman_nums = r'([IVX]+)'

rom = {'I':1, 'V':5, 'X':10}

def roman_to_digits(roman:str) -> int:
    """Only for combinations from 'IVX'."""
    result = 0
    for ind,symbol in enumerate(roman):
        if ind + 1 == len(roman):
            return result + rom[symbol]

        # I < V < X
        if symbol < roman[ind+1]:
            result-= rom[symbol]
        else:
            result+= rom[symbol]

def roman_apply(foundation_year, _foundation_year) -> (str, bool):
    search_res = re.search(roman_nums, foundation_year)
    if search_res is None:
        return foundation_year, _foundation_year

    year = roman_to_digits(search_res.group(1))*100 -1
    # до н. э.
    if 'до' in foundation_year:
        year*= -1

    return str(year), not _foundation_year

In [16]:
data[['foundation_year', '_foundation_year']] = data[['foundation_year', '_foundation_year']].apply(lambda x: roman_apply(*x), axis=1, result_type='expand')

In [17]:
data[data._foundation_year == True].sample(5)

,address,postal_code,country,federal_district,region_type,region,area_type,area,city_type,city,...,capital_marker,okato,oktmo,tax_office,timezone,geo_lat,geo_lon,population,foundation_year,_foundation_year
90,"Тверская обл, г Торжок",172000,Россия,Центральный,обл,Тверская,None,None,город,Торжок,...,0,28450000000,28750000,6915,UTC+3,57.04133,34.9602344,47702,9-10 века,True
63,"Нижегородская обл, г Первомайск",607751,Россия,Приволжский,обл,Нижегородская,None,None,город,Первомайск,...,0,22434000000,22734000001,5254,UTC+3,54.8675792,43.8013992,14567,19 век,True
11,"Волгоградская обл, г Котово",403801,Россия,Южный,обл,Волгоградская,р-н,Котовский,город,Котово,...,1,18226501000,18626101001,3453,UTC+4,50.3205766,44.8030699,24104,1710-е,True
55,"Мурманская обл, г Кандалакша",184042,Россия,Северо-Западный,обл,Мурманская,р-н,Кандалакшский,город,Кандалакша,...,1,47202501000,47608101,5102,UTC+3,67.1567974,32.4143218,35659,11 век,True
100,"Респ Чеченская, г Шали",366300,Россия,Северо-Кавказский,Респ,Чеченская,р-н,Шалинский,город,Шали,...,1,96237501000,96637101001,2000,UTC+3,43.1488691,45.9009629,47715,1378 или 1384,True


In [18]:
def cut_endswith_e(year, flag) -> (str | None, bool):
    if year is None or not flag:
        return year, flag

    if year.endswith('-е'):
        return year[:-2], not flag
    else:
        return year, flag

In [19]:
data[['foundation_year', '_foundation_year']] = data[['foundation_year', '_foundation_year']].apply(lambda x: cut_endswith_e(*x), axis=1, result_type='expand')

In [20]:
data[data._foundation_year == True].sample(5)

,address,postal_code,country,federal_district,region_type,region,area_type,area,city_type,city,...,capital_marker,okato,oktmo,tax_office,timezone,geo_lat,geo_lon,population,foundation_year,_foundation_year
20,"Респ Кабардино-Балкарская, г Майский",361110,Россия,Северо-Кавказский,Респ,Кабардино-Балкарская,р-н,Майский,город,Майский,...,1,83220501000,83620101,716,UTC+3,43.6281516,44.0517314,26755,19 век,True
98,"Респ Чеченская, г Гудермес",366200,Россия,Северо-Кавказский,Респ,Чеченская,р-н,Гудермесский,город,Гудермес,...,1,96404000000,96610101001,2032,UTC+3,43.3519142,46.1035645,45643,19 век,True
102,"Ярославская обл, г Мышкин",152830,Россия,Центральный,обл,Ярославская,р-н,Мышкинский,город,Мышкин,...,1,78221501000,78621101,7612,UTC+3,57.7881684,38.4544224,5932,15 век,True
60,"Нижегородская обл, г Кстово",607650,Россия,Приволжский,обл,Нижегородская,р-н,Кстовский,город,Кстово,...,1,22237501000,22637101001,5250,UTC+3,56.1432084,44.1664198,66641,15 век,True
84,"Респ Татарстан, г Арск",422000,Россия,Приволжский,Респ,Татарстан,р-н,Арский,город,Арск,...,1,92212501000,92612151,1685,UTC+3,56.0912567,49.877067,18114,13 век,True


In [21]:
def cut_endswith_sent(year, flag) -> (str, bool):
    if year.endswith('век'):
        return str(int(year[:-4])*100 - 1), not flag
    else:
        return year, flag

In [22]:
data[['foundation_year', '_foundation_year']] = data[['foundation_year', '_foundation_year']].apply(lambda x: cut_endswith_sent(*x), axis=1, result_type='expand')

In [23]:
data[data._foundation_year == True].foundation_year

36       497 год до н. э.
37     7 — 6 век до н. э.
61              1677-1678
68               8—9 века
90              9-10 века
100         1378 или 1384
Name: foundation_year, dtype: object

In [24]:
data.at[36, 'foundation_year'] = '-497'
data.at[37, 'foundation_year'] = '500'
data.at[61, 'foundation_year'] = '1678'
data.at[68, 'foundation_year'] = '800'
data.at[90, 'foundation_year'] = '900'
data.at[100, 'foundation_year'] = '1384'
data.drop(columns='_foundation_year', inplace=True)

## Save and upload

In [25]:
data.to_csv('handled_1.csv', encoding='utf-8', index=False)

In [26]:
headers = {'Content-Type': 'text/csv'}
response = requests.post(endpoint,
                         data=open('handled_1.csv' ,'rb'),
                         headers=headers)
response.status_code

200